In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import numpy as np
from scipy import stats
import json
import math

In [30]:
symbol="V"
timeInterval = "1d"
timeRange = "1y"

In [3]:
def metadata_price_action(ticker, interval, timeRange):
    stockData = requests.get(f"https://query1.finance.yahoo.com/v8/finance/chart/{ticker}?region=US&lang=en-US&includePrePost=false&interval={interval}&useYfid=true&range={timeRange}&corsDomain=finance.yahoo.com&.tsrc=finance")
    stockJson = stockData.json()
    stockResult = stockJson["chart"]["result"][0]
    stockMetadata = stockResult["meta"]
    stockPriceData = pd.DataFrame({"timestamp": stockResult["timestamp"]})
    stockIndicators = pd.DataFrame(stockResult["indicators"]["quote"][0])
    stockDataMerge = pd.concat([stockPriceData, stockIndicators], axis=1)
    return {"metadata": stockMetadata, "price":stockDataMerge}

In [4]:
 def extended_stock_stats(ticker):
    stockData = requests.get(f"https://finance.yahoo.com/quote/{ticker}/key-statistics?p={ticker}")
    soup = BeautifulSoup(stockData.content, 'html.parser')
    job_elems = soup.find_all('section',  {"data-test":"qsp-statistics"})
    datObj={}
    dat=[]
    for job_elem in job_elems:
        for t in job_elem.find_all("tr"):
            if(len(dat)==2):
                head, *tail=np.array(dat)
                datObj[head]=tail[0]
            elif(len(dat)>=2):
                head, *tail=np.array(dat)
                datObj[head]=tail
            dat=[]
            for td in t.find_all("td"):
                dat.append(td.text)
    return datObj


In [5]:
 def basic_stats(ticker):
    statsData = requests.get(f"https://finance.yahoo.com/quote/{ticker}?p={ticker}&.tsrc=fin-srch")
    statsSoup = BeautifulSoup(statsData.content, 'html.parser')
    stats_elem = statsSoup.find('div',id="quote-summary")
    statsObj={}
    stats=[]
    if stats_elem != None:
        for t in stats_elem.find_all("tr"):
            if(len(stats)==2):
                head, *tail=np.array(stats)
                statsObj[head]=tail[0]
            elif(len(stats)>=2):
                head, *tail=np.array(stats)
                statsObj[head]=tail
            stats=[]
            for td in t.find_all("td"):
                stats.append(td.text)
    return statsObj

In [6]:
def current_sp500_symbols():
    sp500 = requests.get("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
    soup = BeautifulSoup(sp500.content, 'html.parser')
    symbols=[]
    symbol_table = soup.find_all('table', id="constituents")
    for sym in symbol_table:
        for tr in sym.find_all("tr"):
            td = tr.find("td")
            if(td != None):
                symbols.append(td.text.strip())
    return symbols

In [7]:
def parse_value(val):
    lastChar = val[-1]
    number = val[0:-1]
    number = number.replace(",","")
    if(lastChar == "A"):
        return None
    elif (lastChar == "M"):
        return float(number) * 1000000
    elif (lastChar == "B"):
        return float(number) * 1000000000
    elif (lastChar == "T"):
        return float(number) * 1000000000000
    elif (lastChar == "%"):
        return float(number) / 100
    else:
        return float(number)

In [8]:
sp500symbols = np.array(current_sp500_symbols())
sp500symbols

array(['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADBE', 'AMD', 'AAP',
       'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALXN',
       'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN',
       'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP',
       'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'ANTM', 'AON', 'AOS',
       'APA', 'AAPL', 'AMAT', 'APTV', 'ADM', 'ANET', 'AJG', 'AIZ', 'T',
       'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'BKR', 'BLL', 'BAC',
       'BK', 'BAX', 'BDX', 'BRK.B', 'BBY', 'BIO', 'BIIB', 'BLK', 'BA',
       'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BF.B', 'CHRW',
       'COG', 'CDNS', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT',
       'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CERN', 'CF',
       'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS',
       'CSCO', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS', 'KO', 'CTSH',
       'CL', 'CMCSA', 'CMA', 'CAG', 'COP', 'ED', 'STZ', 'COO', 'CPRT',


In [38]:
my_columns = ["Ticker", "Stock Price","4 Week Change", "4 Week Percentile", "12 Week Change", "12 Week Percentile", "26 Week Change", "26 Week Percentile", "52 Week Change", "52 Week Percentile", "HQM Score"]
dataframe = pd.DataFrame(columns=my_columns)
for sym in sp500symbols[:]:
    if(sym.find(".") > -1):
        sym = sym.replace(".", "-")
    basic = basic_stats(sym)
    complexStats = extended_stock_stats(sym)
    previousClose = parse_value(basic["Previous Close"])
    metadataAndPrice = metadata_price_action(sym, "1d", "1y")
    if(len(metadataAndPrice["price"]) >= 250):
        fourWeekChange = (metadataAndPrice["price"].iloc[19]["close"]-metadataAndPrice["price"].iloc[0]["close"])/metadataAndPrice["price"].iloc[0]["close"]
        twelveWeekChange = (metadataAndPrice["price"].iloc[59]["close"]-metadataAndPrice["price"].iloc[0]["close"])/metadataAndPrice["price"].iloc[0]["close"]
        twentySixWeekChange = (metadataAndPrice["price"].iloc[129]["close"]-metadataAndPrice["price"].iloc[0]["close"])/metadataAndPrice["price"].iloc[0]["close"]
        fiftyTwoWeekChange = (metadataAndPrice["price"].iloc[-1]["close"]-metadataAndPrice["price"].iloc[0]["close"])/metadataAndPrice["price"].iloc[0]["close"]
        dataframe = dataframe.append(pd.Series([sym,previousClose,fourWeekChange, "N/A", twelveWeekChange, "N/A", twentySixWeekChange, "N/A", fiftyTwoWeekChange, "N/A", "N/A"], index=my_columns), ignore_index=True)
        print(sym, previousClose, fourWeekChange, twelveWeekChange, twentySixWeekChange, fiftyTwoWeekChange)
# dataframe.sort_values("52 Week Change", ascending=False, inplace=True)
dataframe.reset_index(inplace=True)
dataframe.to_csv("lastBetterMomentumDataframe_bak.csv")
# dataframe = dataframe[:50]
dataframe

MMM 179.3 -0.13584758840686154 -0.14234354448125697 0.03134460959402151 0.11339553484096639
ABT 125.4 -0.10400177530475534 0.03753108443573313 0.14616776440194854 0.39091117102582584
ABBV 106.0 -0.24620613128662938 -0.045526910354415036 0.0029714397678228697 0.11440091685514799
ABMD 322.0 -0.1280937562157913 0.09632319291428615 0.8143146649507489 0.9750713803203201
ACN 257.6 -0.2933613921951185 -0.15853197461861943 0.11150073297831739 0.18882655699931875
ATVI 102.2 -0.1314062476158142 0.13999998569488525 0.2984374761581421 0.5750000476837158
ADBE 488.3 -0.222582879474005 -0.062196478092950896 0.2568608785432414 0.26430231665756815
AMD 88.6 -0.3169198774367819 -0.04819280800016679 0.4506722746113082 0.5641697418375171
AAP 167.5 -0.3771757402810089 -0.1498455861477436 0.10289166270150185 0.16493543447978803
AES 27.8 -0.5403845783521424 -0.4317307630586904 -0.13124998280635183 0.36105771655514485
AFL 47.2 -0.5092250993588212 -0.3637600067270247 -0.2740337958903658 -0.06622644661591023
A 1

,index,Ticker,Stock Price,4 Week Change,4 Week Percentile,12 Week Change,12 Week Percentile,26 Week Change,26 Week Percentile,52 Week Change,52 Week Percentile,HQM Score
0,0,MMM,179.3,-0.135848,N/A,-0.142344,N/A,0.031345,N/A,0.113396,N/A,N/A
1,1,ABT,125.4,-0.104002,N/A,0.037531,N/A,0.146168,N/A,0.390911,N/A,N/A
2,2,ABBV,106.0,-0.246206,N/A,-0.045527,N/A,0.002971,N/A,0.114401,N/A,N/A
3,3,ABMD,322.0,-0.128094,N/A,0.096323,N/A,0.814315,N/A,0.975071,N/A,N/A
4,4,ACN,257.6,-0.293361,N/A,-0.158532,N/A,0.111501,N/A,0.188827,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
497,497,YUM,105.3,-0.415712,N/A,-0.198132,N/A,-0.070376,N/A,0.012130,N/A,N/A
498,498,ZBRA,477.6,-0.235868,N/A,-0.086313,N/A,0.134600,N/A,1.022660,N/A,N/A
499,499,ZBH,156.8,-0.490940,N/A,-0.292253,N/A,-0.136246,N/A,-0.010859,N/A,N/A
500,500,ZION,50.0,-0.406054,N/A,-0.404541,N/A,-0.285622,N/A,0.117838,N/A,N/A


In [27]:
dataframe.to_csv("momentum_almost.csv")

In [45]:
time_periods = [
                '52 Week',
                '26 Week',
                '12 Week',
                '4 Week'
                ]

for row in dataframe.index:
    for time_period in time_periods:
        dataframe.loc[row, f'{time_period} Percentile'] = stats.percentileofscore(dataframe[f'{time_period} Change'], dataframe.loc[row, f'{time_period} Change'])/100

from statistics import mean

for row in dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(dataframe.loc[row, f'{time_period} Percentile'])
    dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
sorted_dataframe = dataframe.sort_values(by = 'HQM Score', ascending = False)
sorted_dataframe.to_csv("momentum_last.csv")

In [33]:
metadataAndPrice = metadata_price_action(symbol, timeInterval, timeRange)
metadata = metadataAndPrice["metadata"]
metadata

{'currency': 'USD',
 'symbol': 'V',
 'exchangeName': 'NYQ',
 'instrumentType': 'EQUITY',
 'firstTradeDate': 1205933400,
 'regularMarketTime': 1613768402,
 'gmtoffset': -18000,
 'timezone': 'EST',
 'exchangeTimezoneName': 'America/New_York',
 'regularMarketPrice': 204.73,
 'chartPreviousClose': 213.31,
 'priceHint': 2,
 'currentTradingPeriod': {'pre': {'timezone': 'EST',
   'start': 1613725200,
   'end': 1613745000,
   'gmtoffset': -18000},
  'regular': {'timezone': 'EST',
   'start': 1613745000,
   'end': 1613768400,
   'gmtoffset': -18000},
  'post': {'timezone': 'EST',
   'start': 1613768400,
   'end': 1613782800,
   'gmtoffset': -18000}},
 'dataGranularity': '1d',
 'range': '1y',
 'validRanges': ['1d',
  '5d',
  '1mo',
  '3mo',
  '6mo',
  '1y',
  '2y',
  '5y',
  '10y',
  'ytd',
  'max']}

In [34]:
price = metadataAndPrice["price"]
price

,timestamp,open,high,volume,low,close
0,1582209000,212.130005,213.690002,8531900,208.279999,211.449997
1,1582295400,209.899994,210.220001,9231500,207.419998,208.809998
2,1582554600,197.889999,203.770004,13316800,194.009995,198.789993
3,1582641000,198.789993,199.889999,18539000,187.720001,188.399994
4,1582727400,190.000000,193.509995,14214700,187.029999,187.210007
...,...,...,...,...,...,...
248,1613140200,209.360001,211.130005,6910600,207.679993,209.960007
249,1613485800,210.660004,211.050003,9651400,207.350006,207.899994
250,1613572200,205.259995,208.279999,8373200,205.059998,207.509995
251,1613658600,205.880005,209.699997,7328600,204.960007,209.350006


In [10]:
basicStats = basic_stats(symbol)
basicStats

{'Previous Close': '120.91',
 'Open': '121.00',
 'Bid': '120.86 x 1000',
 'Ask': '120.91 x 800',
 "Day's Range": '120.09 - 121.36',
 '52 Week Range': '90.56 - 151.89',
 'Volume': '3,879,636',
 'Avg. Volume': '6,234,309',
 'Market Cap': '107.64B',
 'Beta (5Y Monthly)': '1.25',
 'PE Ratio (TTM)': '19.39',
 'EPS (TTM)': '6.23',
 'Earnings Date': 'Apr 19, 2021 - Apr 23, 2021',
 'Forward Dividend & Yield': '6.52 (5.40%)',
 'Ex-Dividend Date': 'Feb 09, 2021'}

In [11]:
extStats = extended_stock_stats(symbol)
extStats

{'Market Cap (intraday) 5': ['107.64B',
  '112.17B',
  '108.41B',
  '107.56B',
  '98.49B',
  '118.91B'],
 'Enterprise Value 3': ['160.30B',
  '166.96B',
  '164.08B',
  '165.14B',
  '157.78B',
  '179.58B'],
 'Trailing P/E ': ['19.71', '14.26', '13.79', '11.96', '10.49', '15.57'],
 'Forward P/E 1': ['10.95', '10.75', '9.98', '11.15', '8.31', '9.92'],
 'PEG Ratio (5 yr expected) 1': ['1.25',
  '9.35',
  '7.73',
  '9.69',
  '3.59',
  '22.56'],
 'Price/Sales (ttm)': ['1.47', '1.50', '1.44', '1.41', '1.28', '1.56'],
 'Price/Book (mrq)': ['5.23', '5.29', '5.28', '5.38', '4.73', '6.62'],
 'Enterprise Value/Revenue 3': ['2.18',
  '8.20',
  '9.34',
  '9.11',
  '8.98',
  '8.25'],
 'Enterprise Value/EBITDA 6': ['12.70',
  '50.52',
  '42.81',
  '46.22',
  '82.57',
  '29.95'],
 'Beta (5Y Monthly) ': '1.25',
 '52-Week Change 3': '-20.05%',
 'S&P500 52-Week Change 3': '16.75%',
 '52 Week High 3': '151.89',
 '52 Week Low 3': '90.56',
 '50-Day Moving Average 3': '124.29',
 '200-Day Moving Average 3': '1